# Task-2

In [22]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import scipy as sp
import sklearn_crfsuite
#importing vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
#Reading train data
train_df = pd.read_csv("aij-wikiner-en-wp2", sep=r'\n', header=None, engine='python')
train_df.columns = ['text']
temp = pd.DataFrame(train_df.text.str.split(' ').tolist()).stack()
train_df = temp.reset_index()
del temp
train_df.columns = ["sentence_id", "no_words_in_sent", "text"]
#Splitting of train data
train_df[['word','pos_tag', 'iob_tag']] = train_df['text'].str.split('|',expand=True)
train_df = train_df[['word','pos_tag','iob_tag']]
print(train_df.head())

         word pos_tag iob_tag
0         The      DT  I-MISC
1      Oxford     NNP  I-MISC
2   Companion     NNP  I-MISC
3          to      TO  I-MISC
4  Philosophy     NNP  I-MISC


In [24]:
#Reading test data
test_data = pd.read_csv("wikigold.conll.txt",sep = "\n\n",header=None, engine ='python', delimiter = ' ')
#Assigning column names
test_data.columns = ['word','iob_tag']
#Calculating pos tag and adding pos column to the test data
test_data['pos'] = nltk.pos_tag(test_data['word'])
test_data['pos_tag'] = test_data['pos'].apply(lambda x:x[1])
#Selecting required columns
test_data = test_data[['word','pos_tag','iob_tag']]
print(test_data.head())

    word pos_tag iob_tag
0    010      CD  I-MISC
1     is     VBZ       O
2    the      DT       O
3  tenth      JJ       O
4  album      NN       O


In [25]:
train_df['end_of_sent'] = train_df['word'].apply(lambda x:1 if x[0]=="." else 0)
train_df['is_upper'] = train_df['word'].apply(lambda x:1 if x[0].isupper() else 0)
train_df['is_NNP'] = (train_df['pos_tag'].shift() == 'NNP')

In [26]:
train_df['is_NNP']  = train_df['is_NNP'].apply(lambda x:1 if x==True else 0)

In [27]:
train_df['iob_tag'] = train_df['iob_tag'].apply(lambda x:'I-PER' if x == 'B-PER'  else x)
train_df['iob_tag'] = train_df['iob_tag'].apply(lambda x:'I-ORG' if x == 'B-ORG'  else x)
train_df['iob_tag'] = train_df['iob_tag'].apply(lambda x:'I-MISC' if x == 'B-MISC'  else x)
train_df['iob_tag'] = train_df['iob_tag'].apply(lambda x:'I-LOC' if x == 'B-LOC'  else x)


In [28]:
train_df.head()

,word,pos_tag,iob_tag,end_of_sent,is_upper,is_NNP
0,The,DT,I-MISC,0,1,0
1,Oxford,NNP,I-MISC,0,1,0
2,Companion,NNP,I-MISC,0,1,1
3,to,TO,I-MISC,0,0,1
4,Philosophy,NNP,I-MISC,0,1,0


In [29]:
test_data['end_of_sent'] = test_data['word'].apply(lambda x:1 if x=="." else 0)
test_data['is_upper'] = test_data['word'].apply(lambda x:1 if x[0].isupper() else 0)
test_data['is_NNP'] = (test_data['pos_tag'].shift() == 'NNP')
test_data['is_NNP']  = test_data['is_NNP'].apply(lambda x:1 if x==True else 0)

In [30]:
test_data.head()

,word,pos_tag,iob_tag,end_of_sent,is_upper,is_NNP
0,010,CD,I-MISC,0,0,0
1,is,VBZ,O,0,0,0
2,the,DT,O,0,0,0
3,tenth,JJ,O,0,0,0
4,album,NN,O,0,0,0


In [31]:
#Creating dummy variable in test as well as train
train_df['dummy'] = train_df['word'] + ' ' + train_df['pos_tag']
test_data['dummy'] = test_data['word'] + ' ' + test_data['pos_tag']

In [32]:
#Fit vectorizer
vectoriser = TfidfVectorizer()
text = list(train_df['dummy'].values) + list(test_data['dummy'].values) 
vectoriser.fit(text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [33]:
old_train = train_df[['end_of_sent','is_upper','is_NNP']]

In [34]:
x_train = sp.sparse.hstack((vectoriser.transform(train_df.dummy),old_train.values),format='csr')

In [35]:
old_test = test_data[['end_of_sent','is_upper','is_NNP']]

In [36]:
x_test_data = sp.sparse.hstack((vectoriser.transform(test_data.dummy),old_test.values),format='csr')

splitting data into train and test:

In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
y = train_df.iob_tag.values
x_train, x_test, y_train, y_test = train_test_split(x_train, y, test_size = 0.2, random_state=0)

In [38]:
classes = np.unique(y)
classes = classes.tolist()

In [39]:
#importing classifier
from sklearn.linear_model import Perceptron
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(x_train, y_train, classes)
print(classification_report(y_pred=per.predict(x_test), y_true=y_test, labels=classes))
print(accuracy_score(y_pred=per.predict(x_test), y_true=y_test))

C:\Users\Chinnari\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1-- Epoch 1
-- Epoch 1

-- Epoch 1-- Epoch 1

Norm: 220.09, NNZs: 27344, Bias: -2.020000, T: 2799724, Avg. loss: 0.018114
Total training time: 1.25 seconds.
Norm: 165.44, NNZs: 16902, Bias: -1.940000, T: 2799724, Avg. loss: 0.025960
Total training time: 1.32 seconds.


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.1s remaining:    3.2s


Norm: 120.17, NNZs: 12706, Bias: 1.020000, T: 2799724, Avg. loss: 0.023239
Total training time: 1.38 seconds.
Norm: 184.07, NNZs: 22682, Bias: -1.510000, T: 2799724, Avg. loss: 0.025662

[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    2.1s remaining:    1.4s



Total training time: 1.39 seconds.
Norm: 189.00, NNZs: 24637, Bias: -2.040000, T: 2799724, Avg. loss: 0.036538
Total training time: 1.44 seconds.


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished


              precision    recall  f1-score   support

       I-LOC       0.85      0.39      0.53     24816
      I-MISC       0.61      0.50      0.55     25920
       I-ORG       0.62      0.55      0.58     17948
       I-PER       0.60      0.96      0.74     28992
           O       0.98      0.99      0.99    602255

   micro avg       0.94      0.94      0.94    699931
   macro avg       0.73      0.68      0.68    699931
weighted avg       0.94      0.94      0.93    699931

0.9374695505699847


In [40]:
#importing SGDClassifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.partial_fit(x_train, y_train, classes);
print(classification_report(y_pred=sgd.predict(x_test), y_true=y_test, labels=classes));
print(accuracy_score(y_pred=sgd.predict(x_test), y_true=y_test))

              precision    recall  f1-score   support

       I-LOC       0.78      0.65      0.70     24816
      I-MISC       0.78      0.44      0.56     25920
       I-ORG       0.73      0.48      0.58     17948
       I-PER       0.65      0.91      0.76     28992
           O       0.98      1.00      0.99    602255

   micro avg       0.95      0.95      0.95    699931
   macro avg       0.79      0.70      0.72    699931
weighted avg       0.95      0.95      0.94    699931

0.9474348185749738


In [41]:
#importing multinomial
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.partial_fit(x_train, y_train, classes);
print(classification_report(y_pred=nb.predict(x_test), y_true=y_test, labels=classes));
print(accuracy_score(y_pred=nb.predict(x_test), y_true=y_test))


              precision    recall  f1-score   support

       I-LOC       0.79      0.74      0.76     24816
      I-MISC       0.77      0.50      0.60     25920
       I-ORG       0.80      0.50      0.61     17948
       I-PER       0.79      0.92      0.85     28992
           O       0.98      1.00      0.99    602255

   micro avg       0.95      0.95      0.95    699931
   macro avg       0.82      0.73      0.76    699931
weighted avg       0.95      0.95      0.95    699931

0.9530982339687769


# Task - 3

In task 2 we have already read the data in test and changed it into a machine learning readable foremat. We are testing againest the test data.

In [42]:
#Pridicating the output
print(classification_report(y_pred=nb.predict(x_test_data),y_true = test_data.iob_tag.values, labels = classes))
print(accuracy_score(y_pred=nb.predict(x_test_data),y_true = test_data.iob_tag.values))

              precision    recall  f1-score   support

       I-LOC       0.63      0.62      0.63       967
      I-MISC       0.55      0.41      0.47       875
       I-ORG       0.73      0.31      0.43      1234
       I-PER       0.57      0.87      0.69      1054
           O       0.97      0.99      0.98     21907

   micro avg       0.92      0.92      0.92     26037
   macro avg       0.69      0.64      0.64     26037
weighted avg       0.92      0.92      0.91     26037

0.9176940507738987
